In [1]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = 'mypassword'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


# Real-time Inventory Management

## Description of the use case
Real-time inventory management involves tracking product quantities and availability across different locations or warehouses. This use case ensures that stock levels are updated instantly as sales and restocks happen, preventing overselling and stockouts.

## Redis Data Structures Used
- **Hashes**: To store product information and stock levels.
- **Streams**: To record real-time updates of stock levels.
- **Sorted Sets**: To manage and query low-stock alerts and prioritize restocking.

In [8]:
if r.exists("product_1"): r.delete("product_1")
if r.exists("product_2"): r.delete("product_2")
if r.exists("product_3"): r.delete("product_3")
if r.exists("stock_updates"): r.delete("stock_updates")
if r.exists("low_stock_alerts"): r.delete("low_stock_alerts")

# Mock data for the use case
mock_data = {
    'product_1': {'name': 'Product 1', 'stock': 100},
    'product_2': {'name': 'Product 2', 'stock': 200},
    'product_3': {'name': 'Product 3', 'stock': 150},
}

# Use Hashes to store product information and stock levels
for product_id, product_info in mock_data.items():
    r.hset(product_id, mapping=product_info)
    print(f"Initial Stock : {product_info} ")

# Use Streams to record real-time updates of stock levels
r.xadd('stock_updates', {'product_id': 'product_1', 'stock_change': -10})
r.xadd('stock_updates', {'product_id': 'product_2', 'stock_change': 20})
r.xadd('stock_updates', {'product_id': 'product_3', 'stock_change': -120})

# Update stock levels based on stream entries
stream_entries = r.xrange('stock_updates')
for entry in stream_entries:
    product_id = entry[1]['product_id']
    stock_change = int(entry[1]['stock_change'])
    current_stock = int(r.hget(product_id, 'stock'))
    new_stock = current_stock + stock_change
    r.hset(product_id, 'stock', new_stock)

# Use Sorted Sets to manage low-stock alerts
for product_id in mock_data.keys():
    stock_level = int(r.hget(product_id, 'stock'))
    if stock_level < 50:
        r.zadd('low_stock_alerts', {product_id: stock_level})

# Print the updated stock levels
for product_id in mock_data.keys():
    print(f"Final Stock : {product_id}: {r.hgetall(product_id)}")

# Print low-stock alerts
low_stock_alerts = r.zrange('low_stock_alerts', 0, -1, withscores=True)
print('Low-stock alerts:')
for product_id, qty in low_stock_alerts:
    print(f"{r.hget(product_id, 'name')} has only {qty} units left")

Initial Stock : {'name': 'Product 1', 'stock': 100} 
Initial Stock : {'name': 'Product 2', 'stock': 200} 
Initial Stock : {'name': 'Product 3', 'stock': 150} 
Final Stock : product_1: {'name': 'Product 1', 'stock': '90'}
Final Stock : product_2: {'name': 'Product 2', 'stock': '220'}
Final Stock : product_3: {'name': 'Product 3', 'stock': '30'}
Low-stock alerts:
Product 3 has only 30.0 units left
